In [1]:
import json
import random
import numpy as np

def extract2():
    '''
    从原来的csv提取出问答对json,将原材料单独标注出来
    保留了question-id
    :return:
    '''
    with open(r"E:\RecipeQA\data\数据集\r2vq_train_10_28_2021\train\crl_srl.csv", 'r', encoding='utf-8') as f:
        all_data = f.readlines()
    dicts = {}
    question = []
    text = []
    meta = []
    ingredients = []
    newdoc_id = ''  # 防止报错
    for i in range(len(all_data) - 1):
        item = all_data[i]
        if 'newdoc id' in item:
            if i != 0:
                dicts[newdoc_id] = {}
                dicts[newdoc_id]['question'] = question
                # text = " ".join(text)
                dicts[newdoc_id]['text'] = text
                dicts[newdoc_id]['meta'] = meta
                dicts[newdoc_id]['ingredients'] = ingredients
            # 清空--
            newdoc_id = item.split("=")[1].strip("\n")
            question = []
            text = []
            meta = []
            ingredients = []
            continue
        item_next = all_data[i + 1]
        if 'question' in item and 'answer' in item_next:
            qa = item.strip("\n") + "    " + item_next.split("=")[1].strip("\n")
            question.append(qa)
        if 'text =' in item:
            item_pre = all_data[i - 1]
            if 'ingredients' in item_pre:
                ingredients.append(item.split("=")[1].strip("\n"))
            else:
                text.append(item.split("=")[1].strip("\n"))
        if 'metadata:url' in item:
            meta.append(item.split('/')[-1].strip("\n"))
    fw = open("datas_1120.json", "w", encoding="utf-8")
    json.dump(dicts, fw, ensure_ascii=False, indent=4)
    fw.close()

# 取出以某id开始的问题
是在extarct()到json文件的基础上提取的。

In [14]:
def extract_id():
    startwith='2'
    texts=[]
    with open(r"datas_1120.json","r",encoding='utf-8') as f:
        data=json.load(f)
    for key,item in data.items():
        questions=item['question']
        for ques in questions:
            q_temp,a=ques.split('     ')
            idx,q = q_temp.split("=")
            idx_=idx.split(" ")[-2]
            if idx_.startswith(startwith):
                text={}
                text['id']=idx_
                text['question']=q
                text['answer']=a
                texts.append(text)
    # fw = open("start3.json", "w", encoding="utf-8")
    # json.dump(texts, fw, ensure_ascii=False, indent=4)
    # fw.close()

# 第2类问题，基本都是第八列的tools或者Habitat

In [68]:
with open(r"E:\RecipeQA\data\数据集\r2vq_train_10_28_2021\train\crl_srl.csv", 'r', encoding='utf-8') as f:
    all_data = f.readlines()
all_data2=[] #存储所有信息
i=1
while i<(219125):
    doc_id = all_data[i-1].split("=")[1].strip("\n") #获取doc_id
    list_2qa=[]   #2代表第二类问题，存储quest和answer
    text_tags=[]  #存储食谱句子和标签
    while 'newdoc id' not in  all_data[i]:
        item = all_data[i]
        item_next = all_data[i + 1]
        if 'question' in item and 'answer' in item_next:
            quest_idx = item.strip().split("=")[0].split(" ")[-2]
            if quest_idx.startswith('2'):
                temp_qa={}
                quest = item.strip().split("=")[1]
                answer = item_next.strip().split("=")[-1]
                temp_qa['quest']=quest
                temp_qa['answer']=answer
                list_2qa.append(temp_qa)
            i+=1
        elif 'text =' in item:
            item_pre = all_data[i - 1]
            if 'ingredients' in item_pre:
                i+=1
                pass
            else:
                i+=1
                temp_text_tags={}
                texts,hiddens="",[]
                while all_data[i] and "sent_id" not in all_data[i] and 'newdoc id' not in all_data[i]: #在数据集的最后一行加了newdoc id
                    if "newpar id" in all_data[i]:
                        i+=1
                        continue
                    item=all_data[i]
                    
                    tags = item.strip().split("\t")
                    if len(tags)!=1:
                        word = tags[1]
                        HIDDEN = tags[7].split("|")[-1]
                        texts+=(" "+word)
                        hiddens.append(HIDDEN)
                    i+=1
                temp_text_tags["text"]=texts
                temp_text_tags["tags"]=hiddens
                text_tags.append(temp_text_tags)
        else:
            i+=1
    i+=1
    
    doc_dict={}
    doc_dict['doc_id']=doc_id
    doc_dict['list_2qa']=list_2qa
    doc_dict['text_tags']=text_tags
    all_data2.append(doc_dict)
fw = open("2_all_data.json", "w", encoding="utf-8")
json.dump(all_data2, fw, ensure_ascii=False, indent=4)
fw.close()

# 第3类问题，
#分为两种：
what's in the <something> 答案在句子里或者第八列的drop下，11列patient可以给予提示
how did you get <something> 答案都是“by 动词ing +第八列tool或habitat” 
但是something一定在第八列的result下

In [7]:
with open(r"E:\RecipeQA\data\数据集\r2vq_train_10_28_2021\train\crl_srl.csv", 'r', encoding='utf-8') as f:
    all_data = f.readlines()
all_data3=[] #存储所有信息
i=1
while i<(219125):
    doc_id = all_data[i-1].split("=")[1].strip("\n") #获取doc_id
    list_3qa=[]   #2代表第二类问题，存储quest和answer
    text_tags=[]  #存储食谱句子和标签
    while 'newdoc id' not in  all_data[i]:
        item = all_data[i]
        item_next = all_data[i + 1]
        if 'question' in item and 'answer' in item_next:
            quest_idx = item.strip().split("=")[0].split(" ")[-2]
            if quest_idx.startswith('3'):
                temp_qa={}
                quest = item.strip().split("=")[1]
                answer = item_next.strip().split("=")[-1]
                temp_qa['quest']=quest
                temp_qa['answer']=answer
                list_3qa.append(temp_qa)
            i+=1
        elif 'text =' in item:
            item_pre = all_data[i - 1]
            if 'ingredients' in item_pre:
                i+=1
                pass
            else:
                i+=1
                temp_text_tags={}
                texts,eight_hiddens,eleve="",[],[]
                while all_data[i] and "sent_id" not in all_data[i] and 'newdoc id' not in all_data[i]: #在数据集的最后一行加了newdoc id
                    if "newpar id" in all_data[i]:
                        i+=1
                        continue
                    item=all_data[i]
                    
                    tags = item.strip().split("\t")
                    if len(tags)!=1:
                        word = tags[1]
                        HIDDEN = tags[7]
                        patient=tags[10]
                        texts+=(" "+word)
                        eight_hiddens.append(HIDDEN)
                        eleve.append(patient)
                    i+=1
                temp_text_tags["text"]=texts
                temp_text_tags["hidden"]=eight_hiddens
                temp_text_tags["patient"]=eleve
                text_tags.append(temp_text_tags)
        else:
            i+=1
    i+=1

    doc_dict={}
    doc_dict['doc_id']=doc_id
    doc_dict['list_2qa']=list_3qa
    doc_dict['text_tags']=text_tags
    all_data3.append(doc_dict)
fw = open("3_all_data.json", "w", encoding="utf-8")
json.dump(all_data3, fw, ensure_ascii=False, indent=4)
fw.close()